In [1]:
import requests
from fake_useragent import UserAgent
from airbyte_cdk.sources import AbstractSource
from airbyte_cdk.sources.streams.http.requests_native_auth import Oauth2Authenticator
from urllib.parse import (
    urljoin,
    urlparse,
    parse_qs
    )

import pendulum

from source_alberta_re.auth import EmailAuthenticator
from source_alberta_re.source import SourceAlbertaRe
from source_alberta_re.streams import Properties, TestStreamConnection

from typing import Mapping, Any

In [12]:
config = {
  "email": "notvalidemail@gmail.com",
  "status": "a",
  "max_days_listed": "1"
}

In [25]:
source_re = SourceAlbertaRe()
streams = source_re.streams(config=config)
stream = streams[0]



{"type": "LOG", "log": {"level": "INFO", "message": "Using email: notvalidemail@gmail.com"}}


In [30]:
stream.request_headers(stream_state=None)

{}

In [26]:
data = stream.read_records(sync_mode=None)

In [9]:
TOKEN_URL = "https://albertare.com/api/leads/register/start"
auth = EmailAuthenticator(
    token_refresh_endpoint=TOKEN_URL,
    email=config["email"]
)

In [33]:
config = {
  "email": "notvalidemail"
}
stream = Properties(config=config)
stream.request_params(stream_state=None)

latest_record = next(stream.read_records(sync_mode=None))

In [37]:
cursor_field = ["attributes", "updated_at"]
if isinstance(cursor_field, str):
    pass
elif isinstance(cursor_field, list):
    for cursor_part in cursor_field:
        latest_record = latest_record.get(cursor_part, {})
    
    cursor_field = cursor_field[-1]



In [38]:
cursor_field

'updated_at'

In [6]:
TOKEN_URL = "https://albertare.com/api/leads/register/start"
auth = EmailAuthenticator(
    token_refresh_endpoint=TOKEN_URL,
    email="thelordjesuschrist@gmail.com"
)

In [7]:
url = "http://albertare.com/api/properties"
headers = {
            "Accept": "application/json",
            "User-Agent": UserAgent(verify_ssl=False).random
        }
headers.update(auth.get_auth_header())

params = {"limit": "1000"}

resp = requests.get(url=url, headers=headers, params=params)

In [11]:
(resp.json()["data"][0])

{'id': '61614801',
 'attributes': {'created_at': '2022-07-07T11:09:37-04:00',
  'updated_at': '2022-07-07T11:09:37-04:00',
  'listed_at': '2022-07-07T11:09:37-04:00',
  'mls': 'A1236164',
  'street': 'Shawnee Drive SW',
  'address': '732 Shawnee Drive SW',
  'city': 'Calgary',
  'state': 'AB',
  'zip': 'T2Y 1V5',
  'neighborhood': 'Shawnee Slopes',
  'area': None,
  'county': 'Calgary',
  'price': 649800,
  'status': 'Active',
  'type': 'Home',
  'description': 'Do not miss your chance to view this beautiful custom built Cardel home in an amazing location! This home has tons of curb appeal with its immaculately kept landscaped lawn. Upon entering the home, you are greeted with a huge family room with vaulted ceilings, a large, south facing bay window and skylights above, allowing for lots of natural light. The kitchen boasts upgraded stainless steel appliances, updated counter tops, built-in wine rack and room for a dining table. The many windows in this space overlook the beautiful ba

In [17]:
resp.json()['meta']['pagination']['current_page'] 

resp.json()['meta']['pagination']['total_pages']

{'total': 9511,
 'count': 100,
 'per_page': 100,
 'current_page': 1,
 'total_pages': 96,
 'links': {'next': 'https://albertare.com/api/properties?limit=1000&page=2'}}